misc imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
from functools import partial

import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

# Config

In [3]:
import torch
import torch.nn as nn

from saltsegm.utils import ratio2groups, dump_json
from saltsegm.metrics import inverse_l1
from saltsegm.dataset import Dataset, filter_notarget_ids
from saltsegm.cross_validation import get_cv_111

# Model selection:
from saltsegm.torch_models.resnet_regression import get_ResReg

In [4]:
DATA_PATH = '/nmnt/x04-hdd/boris_temp/salt_prep/train/'
EXP_PATH = '/nmnt/x04-hdd/boris_temp/experiments/salt_segm/regression/ResReg_baseline/'

task_type = 'other'

modalities = ['image-128']
features = None
target = 'target_ratio'

ds = Dataset(data_path = DATA_PATH, modalities=modalities, features=features, target=target)
ids = ds.ids

n_channels = ds.n_channels
n_classes = 1

# ================================================================================

n_splits = 5
val_size = 100

batch_size = 32
epochs = 100
steps_per_epoch = 100

lr_init = 1e-3
patience = 5
lr_factor = 0.2

optim = partial(torch.optim.Adam, lr=lr_init)

lr_scheduler = partial(torch.optim.lr_scheduler.ReduceLROnPlateau,
                       factor=lr_factor, patience=patience)

n_filters = 16  # hardcoded for a while

# ================================================================================

random_state = 0xBadCafe

split_type = 'tvt'

ratio = ds.metadata['target_ratio'].values
groups = ratio2groups(ratio)

ids = filter_notarget_ids(ids=ids, df=ds.metadata)
groups = filter_notarget_ids(ids=groups, df=ds.metadata)

n_vals_to_carry = 5
cv_splits = get_cv_111(ids=ids, n_splits=n_splits, val_size=val_size,
                       groups=groups, random_state=random_state)[:n_vals_to_carry]

# ================================================================================


load_x = ds.load_x
load_y = ds.load_y

metric_fn = inverse_l1
metrics_dict = {'inverse_l1': inverse_l1}

loss_fn = nn.MSELoss(size_average=True)

## Build experiment

In [5]:
from saltsegm.experiment import generate_experiment

generate_experiment(exp_path=EXP_PATH, cv_splits=cv_splits, dataset=ds,
                    split_type=split_type, task_type=task_type)

In [6]:
from saltsegm.experiment import load_val_data, make_predictions, calculate_metrics
from saltsegm.batch_iter import BatchIter
from saltsegm.torch_models.model import TorchModel, fit_model

from saltsegm.utils import save


for n_val in tqdm(range(n_vals_to_carry)):
    VAL_PATH = os.path.join(EXP_PATH, f'experiment_{n_val}')
    
    x_val, y_val = load_val_data(exp_path=EXP_PATH, n_val=n_val)
    val_data = (x_val, y_val)

    train_ids = cv_splits[n_val]['train_ids']

    batch_iter = BatchIter(train_ids=train_ids, load_x=load_x, load_y=load_y,
                           batch_size=batch_size, verbose_loading=False)
        
    model = get_ResReg(n_channels=n_channels, n_filters=n_filters)
    
    torch_model = TorchModel(model=model, loss_fn=loss_fn, metric_fn=metric_fn,
                             optim=optim, lr_scheduler=lr_scheduler, task_type=task_type)
    
    resources = {
        'torch_model': torch_model,
        'batch_iter': batch_iter,
        'epochs': epochs,
        'steps_per_epoch': steps_per_epoch,
        'saving_model_mode': 'max',
        'metrics_dict': metrics_dict
    }
    
    save(obj=resources, path=VAL_PATH, filename='resources', gz=9)

100%|██████████| 5/5 [01:14<00:00, 15.01s/it]


## Results

In [5]:
from saltsegm.experiment import get_experiment_result
from saltsegm.visualization import plot_metrics

In [ ]:
for metric_name in metrics_dict.keys():
    print(
        metric_name, 
        get_experiment_result(exp_path=EXP_PATH, n_splits=n_splits, metric_name=metric_name)
    )

In [ ]:
for i in range(n_vals_to_carry):
    print(f'===  experiment {i}  ===')
    plot_metrics(EXP_PATH, n_val=i)

<hr/>